In this notebook we cleaned up the data on 03/05/2025.

In [1]:
import numpy as np
import pandas as pd


In [2]:
housing = pd.read_csv('cal_fire_data.csv')
housing.info()


/var/folders/l1/qgkmnkpx453gnydcmvkyxhh80000gn/T/ipykernel_44520/3926430974.py:1: DtypeWarning: Columns (12,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  housing = pd.read_csv('cal_fire_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130722 entries, 0 to 130721
Data columns (total 46 columns):
 #   Column                                                        Non-Null Count   Dtype  
---  ------                                                        --------------   -----  
 0   OBJECTID                                                      130722 non-null  int64  
 1   * Damage                                                      130722 non-null  object 
 2   * Street Number                                               126302 non-null  float64
 3   * Street Name                                                 125236 non-null  object 
 4   * Street Type (e.g. road, drive, lane, etc.)                  116260 non-null  object 
 5   Street Suffix (e.g. apt. 23, blding C)                        62017 non-null   object 
 6   * City                                                        98991 non-null   object 
 7   State                                                   

In [3]:
#We start by removing the colums that had unusable or repeated information.

housing = housing.drop(columns=['GLOBALID', 'OBJECTID', 'Zip Code', '* Street Name',
                                '* Street Number', 'Site Address (parcel)', 'x', 'y',
                                'Incident Number (e.g. CAAEU 123456)', 'Battalion',
                                'State', 'APN (parcel)'])



#We removed the 2 rows where the incident was not Fire, and then the column Hazard Type.

housing.drop(housing.index[housing['Hazard Type'] != 'Fire'], inplace = True)
housing = housing.drop(columns=['Hazard Type'])


In [4]:
# We removed the rows where Fire Damage was Inaccessible. We created a new row copying assessing damage with numerical values instead.

housing.drop(housing.index[housing['* Damage'] == 'Inaccessible'], inplace = True)
housing['Damage_Level'] = housing['* Damage'].replace({'No Damage': 0, 'Affected (1-9%)': 1, 'Minor (10-25%)': 1, 'Major (26-50%)': 1, 'Destroyed (>50%)': 1})
housing = housing.drop(columns=['* Damage'])


/var/folders/l1/qgkmnkpx453gnydcmvkyxhh80000gn/T/ipykernel_44520/864498600.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  housing['Damage_Level'] = housing['* Damage'].replace({'No Damage': 0, 'Affected (1-9%)': 1, 'Minor (10-25%)': 1, 'Major (26-50%)': 1, 'Destroyed (>50%)': 1})


In [5]:
# We imputed missing Year Built values
from sklearn.impute import KNNImputer

def missingyear(year):
    if year < 1000.0:
        return np.nan
    return year

housing['Year Built (parcel)'] = housing['Year Built (parcel)'].apply(missingyear)
impute = KNNImputer(missing_values=np.nan)
housing['Year Built (parcel)'] = impute.fit_transform(np.array(housing['Year Built (parcel)']).reshape(-1, 1))


In [6]:
# We created a new row with the building's age
housing['Age'] = 2025 - housing['Year Built (parcel)']
housing = housing.drop(columns=['Year Built (parcel)'])


In [7]:
#The column Incident Start Date includes badly recorded hours, so we cleaned it up to only include a date.
# The date now always appears in DD/MM/YYYY format.

def cleandate(date):
    date = date[:date.find(' ')]
    month = date[:date.find('/')]
    year = date[-4:]
    day = date[date.find('/')+1:-5]
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    return(month + '/' + day + '/' + year) 

housing['Incident Start Date'] = housing['Incident Start Date'].apply(cleandate)


In [8]:
#We make a new column that has the date without the day.

def dayremove(date):
    month = date[:date.find('/')]
    year = date[-4:]
    return(month + '/' + year)

housing['Incident Month/Year'] = housing['Incident Start Date'].apply(dayremove)
housing = housing.drop(columns=['Incident Start Date'])


In [9]:
# We remove rows where the incident occurred before 2018, since incidents without damage weren't reported before then and it could bias the data
def getyear(date):
    return int(date[-2:])

housing['Year'] = housing['Incident Month/Year'].apply(getyear)
housing = housing[housing['Year'] > 17]
housing = housing.drop(columns=['Year'])


In [10]:
housing.to_csv('Housing_Data_Cleaned_03052025.csv')
